# Hero Overall Win Rates

Simple calculation of the overall win rate for each zero for a given database file.

## TODO:
- Add support for filtering by patch

In [11]:
import sqlite3
import os
import meta
import json
import pandas as pd
import datetime as dt

DB_FILE="matches_1_all_2020032410.db"

In [2]:
conn=sqlite3.connect(DB_FILE)

In [3]:
c=conn.cursor()

In [4]:
c.execute("SELECT start_time, radiant_heroes, dire_heroes, radiant_win FROM {}".format(os.environ['DOTA_SQL_STATS_TABLE']))
rows=c.fetchall()

In [7]:
winner_list=[]
total_list=[]

min_start_time=None
max_start_time=None

for row in rows:
    
    # Radiant win
    if row[3]==1:
        for hero in json.loads(row[1]):
            winner_list.append(meta.HERO_DICT[hero])
            total_list.append(meta.HERO_DICT[hero])
        for hero in json.loads(row[2]):
            total_list.append(meta.HERO_DICT[hero])            
    
    # Dire win
    if row[2]==1:
        for hero in json.loads(row[2]):
            winner_list.append(meta.HERO_DICT[hero])
            total_list.append(meta.HERO_DICT[hero])
        for hero in json.loads(row[1]):
            total_list.append(meta.HERO_DICT[hero])
            
    # Update min and max start times
    if min_start_time==None:
        min_start_time=row[0]
        max_start_time=row[0]
    else:
        if row[0]<min_start_time:
            min_start_time=row[0]
        if row[0]>max_start_time:
            max_start_time=row[0]

In [8]:
df_win=pd.DataFrame(winner_list, columns=['hero'])
df_win['win']=1
df_win=df_win.groupby("hero").count()

df_total=pd.DataFrame(total_list, columns=['hero'])
df_total['total']=1
df_total=df_total.groupby("hero").count()

In [9]:
df_overall=df_win.join(df_total)
df_overall['win_rate']=df_overall['win']/df_overall['total']*100

In [10]:
df_overall=df_overall.sort_values(by='win_rate', ascending=False)
for idx,row in df_overall.iterrows():
    print("{0:20} {1:7.2f} {2:6d} {3:6d}".format(
        idx, row['win_rate'], int(row['win']), int(row['total'])))

meepo                  62.93   2905   4616
underlord              58.02   3274   5643
wraith-king            58.01   8091  13947
bloodseeker            55.65   4896   8798
jakiro                 55.02   4253   7730
vengeful-spirit        54.64   2693   4929
ogre-magi              54.48  10387  19064
ursa                   54.30   4548   8375
lycan                  54.08   1585   2931
sniper                 54.02   8681  16071
troll-warlord          53.90   4049   7512
luna                   53.81   3158   5869
crystal-maiden         53.75   6008  11178
dragon-knight          53.55   3305   6172
necrophos              53.04   3823   7208
warlock                52.98   2054   3877
naga-siren             52.74   1778   3371
lone-druid             52.74   1453   2755
clockwerk              52.58   2893   5502
dark-seer              52.55   1144   2177
visage                 52.52    647   1232
axe                    52.38   5695  10873
bounty-hunter          52.37   4644   8867
dazzle     

In [18]:
print("Starting time: {}".format(dt.datetime.utcfromtimestamp(min_start_time).isoformat()))
print("Ending time:   {}".format(dt.datetime.utcfromtimestamp(max_start_time).isoformat()))

Starting time: 2020-03-24T13:31:52
Ending time:   2020-03-26T06:57:14
